In [1]:
from sqlalchemy import create_engine, text
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
from sklearn.metrics.pairwise import haversine_distances
from math import radians, sin, cos, sqrt, atan2
from geopy.distance import geodesic
import fiona
from shapely.geometry import Point
from geopy.geocoders import Nominatim

all_uni_locations Sourced: https://hifld-geoplatform.opendata.arcgis.com/datasets/geoplatform::colleges-and-universities/about
hbcus Sourced: https://nces.ed.gov/ipeds/datacenter/InstitutionByName.aspx?goToReportId=1&sid=1214ca4d-d341-4739-af2b-bb571778daae&rtid=1

In [2]:
all_uni_locations= pd.read_csv('../data/Colleges_and_Universities.csv')
hbcus= pd.read_csv('../data/hbcu_list.csv')

In [3]:
hbcu_merged=all_uni_locations.merge(hbcus, left_on='IPEDSID', right_on='unitid')
hbcu_merged

,X,Y,OBJECTID,IPEDSID,NAME,ADDRESS,CITY,STATE,ZIP,ZIP4,...,TOT_EMP,SHELTER_ID,unitid,institution_name,year,HD2022.Historically Black College or University,agency_id,campus_agency,city_agency,county_agency
0,-8.425464e+06,4.609303e+06,22,163338,UNIVERSITY OF MARYLAND EASTERN SHORE,J.T. WILLIAMS HALL,PRINCESS ANNE,MD,21853,1299,...,765,NOT AVAILABLE,163338,University of Maryland Eastern Shore,2022,Yes,8029.0,y,n,n
1,-9.802458e+06,3.593069e+06,27,102030,BISHOP STATE COMMUNITY COLLEGE,351 NORTH BROAD STREET,MOBILE,AL,36603,5898,...,290,NOT AVAILABLE,102030,Bishop State Community College,2022,Yes,23089.0,y,n,n
2,-9.382946e+06,3.558622e+06,146,133650,FLORIDA AGRICULTURAL AND MECHANICAL UNIVERSITY,1500 S MARTIN LUTHER KING JR BLVD,TALLAHASSEE,FL,32307,NOT AVAILABLE,...,1830,NOT AVAILABLE,133650,Florida Agricultural and Mechanical University,2022,Yes,2920.0,n,y,n
3,-1.095974e+07,3.428517e+06,325,227854,ST PHILIP'S COLLEGE,1801 MARTIN LUTHER KING DR,SAN ANTONIO,TX,78203,2098,...,836,NOT AVAILABLE,227854,St Philip's College,2022,Yes,19871.0,n,y,n
4,-8.407892e+06,4.856108e+06,413,211608,CHEYNEY UNIVERSITY OF PENNSYLVANIA,1837 UNIVERSITY CIRCLE,CHEYNEY,PA,19319,0200,...,136,NOT AVAILABLE,211608,Cheyney University of Pennsylvania,2022,Yes,16403.0,n,n,y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,-8.752139e+06,4.271082e+06,6089,199582,SAINT AUGUSTINE'S UNIVERSITY,1315 OAKWOOD AVENUE,RALEIGH,NC,27610,2298,...,295,NOT AVAILABLE,199582,Saint Augustine's University,2022,Yes,11607.0,n,y,n
97,-8.782952e+06,4.297077e+06,6200,199157,NORTH CAROLINA CENTRAL UNIVERSITY,1801 FAYETTEVILLE STREET,DURHAM,NC,27707,NOT AVAILABLE,...,1404,NOT AVAILABLE,199157,North Carolina Central University,2022,Yes,11118.0,y,n,n
98,-1.044562e+07,3.835148e+06,6204,160649,SOUTHERN UNIVERSITY AT SHREVEPORT,3050 MARTIN LUTHER KING DR,SHREVEPORT,LA,71107,NOT AVAILABLE,...,338,NOT AVAILABLE,160649,Southern University at Shreveport,2022,Yes,7171.0,n,n,y
99,-8.930586e+06,4.312610e+06,6218,199999,WINSTON-SALEM STATE UNIVERSITY,601 MARTIN LUTHER KING JR DR,WINSTON-SALEM,NC,27110,0001,...,951,NOT AVAILABLE,199999,Winston-Salem State University,2022,Yes,11135.0,n,y,n


In [5]:
hbcu_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 53 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   X                                                101 non-null    float64
 1   Y                                                101 non-null    float64
 2   OBJECTID                                         101 non-null    int64  
 3   IPEDSID                                          101 non-null    int64  
 4   NAME                                             101 non-null    object 
 5   ADDRESS                                          101 non-null    object 
 6   CITY                                             101 non-null    object 
 7   STATE                                            101 non-null    object 
 8   ZIP                                              101 non-null    int64  
 9   ZIP4                            

In [8]:
hbcu_locs=folium.Map(location=[30.4275125,	-84.28844136], zoom_start=5, min_zoom=4, max_zoom=25)
for row_index, row_values in hbcu_merged.iterrows():
    loc = [row_values['LATITUDE'], row_values['LONGITUDE']]
    pop = str(row_values['institution_name'])
    icon=folium.Icon(color="black",icon="fa-solid fa-building-columns" , prefix='fa')
    
    marker = folium.Marker(
        location = loc, 
        popup = pop,
        icon = icon) 

    marker.add_to(hbcu_locs)
hbcu_locs.save('../maps/hbcu_locs.html')

hbcu_locs

In [9]:
school_stats= pd.read_csv('../data/school_data/Statistics.csv')
school_stats

,Variable,AVG,Unnamed: 2
0,Average net price-students awarded grant or sc...,"11,229",NaN
1,Number of students receiving a Doctor's degree...,34,NaN
2,Number of students receiving a Master's degree...,84,NaN
3,Number of students receiving a Bachelor's degr...,363,NaN
4,Number of students receiving an Associate's de...,51,NaN
5,Percent of full-time first-time undergraduates...,77,NaN
6,Average amount of federal grant aid awarded to...,"6,705",NaN
7,Percent of full-time first-time undergraduates...,71,NaN
8,Average amount of Pell grant aid awarded to fu...,"5,470",NaN
9,Percent of full-time first-time undergraduates...,44,NaN


In [10]:
outcomes= pd.read_csv('../data/school_data/outcome_statistics.csv')
outcomes

,Variable,AVG,Unnamed: 2
0,Percent full-time first-time receiving an awar...,19,NaN
1,Percent full-time first-time receiving an awar...,32,NaN
2,Percent full-time first-time receiving an awar...,33,NaN
3,Percent full-time first-time still enrolled at...,1,NaN
4,Percent full-time first-time subsequently enro...,26,NaN
5,Percent full-time first-time receiving an asso...,6,NaN
6,Percent full-time first-time receiving a bache...,33,NaN
7,"Percent full-time first-time, Pell grant recip...",17,NaN
8,"Percent full-time first-time, Pell grant recip...",30,NaN
9,"Percent full-time first-time, Pell grant recip...",31,NaN


In [12]:
school_grads= pd.read_csv('../data/school_data/graduation_by_school.csv')
school_grads

,UnitID,Institution Name,Graduation rate total cohort (DRVGR2021_RV),Graduation rate - Bachelor degree within 4 years total (DRVGR2021_RV),Graduation rate - Bachelor degree within 5 years total (DRVGR2021_RV),Graduation rate - Bachelor degree within 6 years total (DRVGR2021_RV),Transfer-out rate - Bachelor cohort (DRVGR2021_RV),Pell Grant recipients - Bachelor's degree rate within 6 years (DRVGR2021_RV),Subsidized Stafford Loan recipients not receiving Pell Grants - Overall graduation rate within 150 percent of normal time (DRVGR2021_RV),Subsidized Stafford Loan recipients not receiving Pell Grants - Bachelor's degree rate within 6 years (DRVGR2021_RV),Did not receive Pell Grants or Subsidized Stafford Loans - Overall graduation rate within 150 percent of normal time (DRVGR2021_RV),Did not receive Pell Grants or Subsidized Stafford Loans - Bachelor's degree rate within 6 years (DRVGR2021_RV),Unnamed: 12
0,100654,Alabama A & M University,28.0,11.0,24.0,28.0,34.0,28.0,32.0,32.0,27.0,27.0,NaN
1,100724,Alabama State University,28.0,10.0,23.0,28.0,36.0,26.0,40.0,40.0,32.0,32.0,NaN
2,138716,Albany State University,24.0,11.0,30.0,35.0,39.0,34.0,27.0,39.0,27.0,36.0,NaN
3,175342,Alcorn State University,42.0,33.0,41.0,42.0,NaN,38.0,38.0,38.0,65.0,65.0,NaN
4,217624,Allen University,23.0,9.0,20.0,23.0,17.0,22.0,38.0,38.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,237899,West Virginia State University,30.0,18.0,28.0,30.0,20.0,29.0,14.0,14.0,37.0,37.0,NaN
97,206491,Wilberforce University,18.0,7.0,16.0,18.0,NaN,19.0,25.0,25.0,11.0,11.0,NaN
98,229887,Wiley College,27.0,13.0,24.0,27.0,39.0,23.0,50.0,50.0,67.0,67.0,NaN
99,199999,Winston-Salem State University,46.0,28.0,44.0,46.0,26.0,44.0,53.0,53.0,53.0,53.0,NaN


In [13]:
school_grads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 13 columns):
 #   Column                                                                                                                                    Non-Null Count  Dtype  
---  ------                                                                                                                                    --------------  -----  
 0   UnitID                                                                                                                                    101 non-null    int64  
 1   Institution Name                                                                                                                          101 non-null    object 
 2   Graduation rate  total cohort (DRVGR2021_RV)                                                                                              98 non-null     float64
 3   Graduation rate - Bachelor degree within 4 years  total

In [19]:
top_10_4yr_grad_rates = school_grads.nlargest(10, school_grads.columns[3])
top_10_4yr_grad_rates

,UnitID,Institution Name,Graduation rate total cohort (DRVGR2021_RV),Graduation rate - Bachelor degree within 4 years total (DRVGR2021_RV),Graduation rate - Bachelor degree within 5 years total (DRVGR2021_RV),Graduation rate - Bachelor degree within 6 years total (DRVGR2021_RV),Transfer-out rate - Bachelor cohort (DRVGR2021_RV),Pell Grant recipients - Bachelor's degree rate within 6 years (DRVGR2021_RV),Subsidized Stafford Loan recipients not receiving Pell Grants - Overall graduation rate within 150 percent of normal time (DRVGR2021_RV),Subsidized Stafford Loan recipients not receiving Pell Grants - Bachelor's degree rate within 6 years (DRVGR2021_RV),Did not receive Pell Grants or Subsidized Stafford Loans - Overall graduation rate within 150 percent of normal time (DRVGR2021_RV),Did not receive Pell Grants or Subsidized Stafford Loans - Bachelor's degree rate within 6 years (DRVGR2021_RV),Unnamed: 12
79,141060,Spelman College,76.0,66.0,73.0,76.0,14.0,73.0,76.0,76.0,84.0,84.0,NaN
35,131520,Howard University,64.0,53.0,62.0,64.0,23.0,58.0,68.0,68.0,69.0,69.0,NaN
33,232265,Hampton University,57.0,40.0,54.0,57.0,31.0,47.0,67.0,67.0,64.0,62.0,NaN
22,158802,Dillard University,52.0,40.0,49.0,52.0,NaN,50.0,62.0,62.0,62.0,62.0,NaN
15,217873,Claflin University,51.0,38.0,48.0,51.0,25.0,47.0,54.0,54.0,78.0,78.0,NaN
26,220181,Fisk University,46.0,38.0,44.0,45.0,28.0,38.0,36.0,36.0,67.0,65.0,NaN
100,160904,Xavier University of Louisiana,50.0,37.0,46.0,48.0,35.0,40.0,44.0,46.0,70.0,69.0,NaN
53,140553,Morehouse College,47.0,33.0,43.0,47.0,31.0,40.0,52.0,52.0,60.0,60.0,NaN
3,175342,Alcorn State University,42.0,33.0,41.0,42.0,NaN,38.0,38.0,38.0,65.0,65.0,NaN
20,130934,Delaware State University,47.0,33.0,44.0,47.0,9.0,45.0,51.0,51.0,51.0,51.0,NaN


In [21]:
top_10_5yr_grad_rates = school_grads.nlargest(10, school_grads.columns[4])
top_10_5yr_grad_rates

,UnitID,Institution Name,Graduation rate total cohort (DRVGR2021_RV),Graduation rate - Bachelor degree within 4 years total (DRVGR2021_RV),Graduation rate - Bachelor degree within 5 years total (DRVGR2021_RV),Graduation rate - Bachelor degree within 6 years total (DRVGR2021_RV),Transfer-out rate - Bachelor cohort (DRVGR2021_RV),Pell Grant recipients - Bachelor's degree rate within 6 years (DRVGR2021_RV),Subsidized Stafford Loan recipients not receiving Pell Grants - Overall graduation rate within 150 percent of normal time (DRVGR2021_RV),Subsidized Stafford Loan recipients not receiving Pell Grants - Bachelor's degree rate within 6 years (DRVGR2021_RV),Did not receive Pell Grants or Subsidized Stafford Loans - Overall graduation rate within 150 percent of normal time (DRVGR2021_RV),Did not receive Pell Grants or Subsidized Stafford Loans - Bachelor's degree rate within 6 years (DRVGR2021_RV),Unnamed: 12
79,141060,Spelman College,76.0,66.0,73.0,76.0,14.0,73.0,76.0,76.0,84.0,84.0,NaN
35,131520,Howard University,64.0,53.0,62.0,64.0,23.0,58.0,68.0,68.0,69.0,69.0,NaN
33,232265,Hampton University,57.0,40.0,54.0,57.0,31.0,47.0,67.0,67.0,64.0,62.0,NaN
22,158802,Dillard University,52.0,40.0,49.0,52.0,NaN,50.0,62.0,62.0,62.0,62.0,NaN
27,133650,Florida Agricultural and Mechanical University,55.0,28.0,49.0,55.0,22.0,53.0,63.0,63.0,61.0,61.0,NaN
15,217873,Claflin University,51.0,38.0,48.0,51.0,25.0,47.0,54.0,54.0,78.0,78.0,NaN
58,199102,North Carolina A & T State University,53.0,29.0,48.0,53.0,24.0,50.0,57.0,57.0,59.0,59.0,NaN
59,199157,North Carolina Central University,51.0,29.0,47.0,51.0,23.0,50.0,55.0,55.0,51.0,51.0,NaN
60,101912,Oakwood University,49.0,27.0,47.0,49.0,NaN,40.0,51.0,51.0,69.0,69.0,NaN
100,160904,Xavier University of Louisiana,50.0,37.0,46.0,48.0,35.0,40.0,44.0,46.0,70.0,69.0,NaN


In [22]:
top_10_6yr_grad_rates = school_grads.nlargest(10, school_grads.columns[5])
top_10_6yr_grad_rates

,UnitID,Institution Name,Graduation rate total cohort (DRVGR2021_RV),Graduation rate - Bachelor degree within 4 years total (DRVGR2021_RV),Graduation rate - Bachelor degree within 5 years total (DRVGR2021_RV),Graduation rate - Bachelor degree within 6 years total (DRVGR2021_RV),Transfer-out rate - Bachelor cohort (DRVGR2021_RV),Pell Grant recipients - Bachelor's degree rate within 6 years (DRVGR2021_RV),Subsidized Stafford Loan recipients not receiving Pell Grants - Overall graduation rate within 150 percent of normal time (DRVGR2021_RV),Subsidized Stafford Loan recipients not receiving Pell Grants - Bachelor's degree rate within 6 years (DRVGR2021_RV),Did not receive Pell Grants or Subsidized Stafford Loans - Overall graduation rate within 150 percent of normal time (DRVGR2021_RV),Did not receive Pell Grants or Subsidized Stafford Loans - Bachelor's degree rate within 6 years (DRVGR2021_RV),Unnamed: 12
17,217891,Clinton College,38.0,0.0,40.0,80.0,NaN,75.0,50.0,NaN,100.0,100.0,NaN
79,141060,Spelman College,76.0,66.0,73.0,76.0,14.0,73.0,76.0,76.0,84.0,84.0,NaN
78,228486,Southwestern Christian College,11.0,33.0,33.0,67.0,NaN,0.0,20.0,100.0,3.0,100.0,NaN
35,131520,Howard University,64.0,53.0,62.0,64.0,23.0,58.0,68.0,68.0,69.0,69.0,NaN
33,232265,Hampton University,57.0,40.0,54.0,57.0,31.0,47.0,67.0,67.0,64.0,62.0,NaN
87,102377,Tuskegee University,57.0,27.0,44.0,57.0,NaN,65.0,63.0,63.0,51.0,51.0,NaN
27,133650,Florida Agricultural and Mechanical University,55.0,28.0,49.0,55.0,22.0,53.0,63.0,63.0,61.0,61.0,NaN
58,199102,North Carolina A & T State University,53.0,29.0,48.0,53.0,24.0,50.0,57.0,57.0,59.0,59.0,NaN
22,158802,Dillard University,52.0,40.0,49.0,52.0,NaN,50.0,62.0,62.0,62.0,62.0,NaN
15,217873,Claflin University,51.0,38.0,48.0,51.0,25.0,47.0,54.0,54.0,78.0,78.0,NaN


In [23]:
top_10_overall_grad_rates = school_grads.nlargest(10, school_grads.columns[2])
top_10_overall_grad_rates

,UnitID,Institution Name,Graduation rate total cohort (DRVGR2021_RV),Graduation rate - Bachelor degree within 4 years total (DRVGR2021_RV),Graduation rate - Bachelor degree within 5 years total (DRVGR2021_RV),Graduation rate - Bachelor degree within 6 years total (DRVGR2021_RV),Transfer-out rate - Bachelor cohort (DRVGR2021_RV),Pell Grant recipients - Bachelor's degree rate within 6 years (DRVGR2021_RV),Subsidized Stafford Loan recipients not receiving Pell Grants - Overall graduation rate within 150 percent of normal time (DRVGR2021_RV),Subsidized Stafford Loan recipients not receiving Pell Grants - Bachelor's degree rate within 6 years (DRVGR2021_RV),Did not receive Pell Grants or Subsidized Stafford Loans - Overall graduation rate within 150 percent of normal time (DRVGR2021_RV),Did not receive Pell Grants or Subsidized Stafford Loans - Bachelor's degree rate within 6 years (DRVGR2021_RV),Unnamed: 12
79,141060,Spelman College,76.0,66.0,73.0,76.0,14.0,73.0,76.0,76.0,84.0,84.0,NaN
35,131520,Howard University,64.0,53.0,62.0,64.0,23.0,58.0,68.0,68.0,69.0,69.0,NaN
33,232265,Hampton University,57.0,40.0,54.0,57.0,31.0,47.0,67.0,67.0,64.0,62.0,NaN
87,102377,Tuskegee University,57.0,27.0,44.0,57.0,NaN,65.0,63.0,63.0,51.0,51.0,NaN
27,133650,Florida Agricultural and Mechanical University,55.0,28.0,49.0,55.0,22.0,53.0,63.0,63.0,61.0,61.0,NaN
58,199102,North Carolina A & T State University,53.0,29.0,48.0,53.0,24.0,50.0,57.0,57.0,59.0,59.0,NaN
22,158802,Dillard University,52.0,40.0,49.0,52.0,NaN,50.0,62.0,62.0,62.0,62.0,NaN
15,217873,Claflin University,51.0,38.0,48.0,51.0,25.0,47.0,54.0,54.0,78.0,78.0,NaN
59,199157,North Carolina Central University,51.0,29.0,47.0,51.0,23.0,50.0,55.0,55.0,51.0,51.0,NaN
100,160904,Xavier University of Louisiana,50.0,37.0,46.0,48.0,35.0,40.0,44.0,46.0,70.0,69.0,NaN


In [24]:
hbcu_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 53 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   X                                                101 non-null    float64
 1   Y                                                101 non-null    float64
 2   OBJECTID                                         101 non-null    int64  
 3   IPEDSID                                          101 non-null    int64  
 4   NAME                                             101 non-null    object 
 5   ADDRESS                                          101 non-null    object 
 6   CITY                                             101 non-null    object 
 7   STATE                                            101 non-null    object 
 8   ZIP                                              101 non-null    int64  
 9   ZIP4                            